In [4]:
import requests
import pandas as pd
from datetime import datetime

# Define the BLS API endpoint and parameters
BLS_API_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

# Example series IDs for unemployment rate (these need to be updated with actual series IDs)
local_area_series_id = "SMU19197802023800001"
state_series_id = "CEU0800000003"

# Define the headers for the API request
headers = {
    'Content-type': 'application/json'
}

# Define the payload for the API request
payload = {
    "seriesid": [local_area_series_id, state_series_id],
    "startyear": "2024",
    "endyear": str(datetime.now().year),
    "registrationkey": "1260ca3e97a44bbca843cb3a6cb500b3"  # Replace with your actual BLS API key
}

# Function to fetch data from BLS API
def fetch_bls_data(payload):
    response = requests.post(BLS_API_URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

# Fetch the data
bls_data = fetch_bls_data(payload)

# Check if data is fetched successfully
if bls_data:
    print("Data fetched successfully.")
else:
    print("Failed to fetch data.")


Data fetched successfully.


In [9]:
# Automatically load all columns based on available data in bls_data

def convert_bls_data_to_df_auto(bls_data):
    """Convert BLS JSON data into DataFrames with all available columns."""
    if bls_data and 'Results' in bls_data:
        series_data = bls_data['Results']['series']
        
        # Dictionary to hold DataFrames for each series
        dataframes = {}
        
        for series in series_data:
            series_id = series['seriesID']
            data = series['data']
            
            # Extract all available fields
            records = []
            for entry in data:
                record = {key: entry[key] for key in entry}
                
                # Convert period to a proper date if it's a monthly period
                if 'period' in record and record['period'].startswith('M'):
                    month = record['period'][1:]  # Remove the 'M' prefix
                    record['date'] = f"{record['year']}-{month}-01"  # Construct a date string
                    record['date'] = pd.to_datetime(record['date'])  # Convert to datetime
                
                records.append(record)
            
            # Create a DataFrame for this series
            df = pd.DataFrame(records)
            
            # Store the DataFrame in the dictionary
            dataframes[series_id] = df
        
        return dataframes
    else:
        print("No data available or incorrect response format.")
        return None

# Convert the BLS data into DataFrames with all columns
bls_dataframes_auto = convert_bls_data_to_df_auto(bls_data)

# Access the DataFrames for each series ID
local_area_df_auto = bls_dataframes_auto.get(local_area_series_id)
state_df_auto = bls_dataframes_auto.get(state_series_id)

# Display the DataFrames
print("Local Area Unemployment Data (All Columns):")
print(local_area_df_auto.head())

print("\nState Unemployment Data (All Columns):")
print(state_df_auto.head())


Local Area Unemployment Data (All Columns):
   year period periodName latest value  \
0  2025    M07       July   true  19.4   
1  2025    M06       June    NaN  19.3   
2  2025    M05        May    NaN  18.8   
3  2025    M04      April    NaN  18.2   
4  2025    M03      March    NaN  17.1   

                                footnotes       date  
0  [{'code': 'P', 'text': 'Preliminary'}] 2025-07-01  
1                                    [{}] 2025-06-01  
2                                    [{}] 2025-05-01  
3                                    [{}] 2025-04-01  
4                                    [{}] 2025-03-01  

State Unemployment Data (All Columns):
   year period periodName latest  value  \
0  2025    M07       July   true  36.05   
1  2025    M06       June    NaN  36.12   
2  2025    M05        May    NaN  35.96   
3  2025    M04      April    NaN  36.00   
4  2025    M03      March    NaN  36.09   

                                footnotes       date  
0  [{'code': 'P', '

In [10]:
local_area_df_auto

,year,period,periodName,latest,value,footnotes,date
0,2025,M07,July,true,19.4,"[{'code': 'P', 'text': 'Preliminary'}]",2025-07-01
1,2025,M06,June,NaN,19.3,[{}],2025-06-01
2,2025,M05,May,NaN,18.8,[{}],2025-05-01
3,2025,M04,April,NaN,18.2,[{}],2025-04-01
4,2025,M03,March,NaN,17.1,[{}],2025-03-01
5,2025,M02,February,NaN,15.8,[{}],2025-02-01
6,2025,M01,January,NaN,16.1,[{}],2025-01-01
7,2024,M12,December,NaN,17.3,[{}],2024-12-01
8,2024,M11,November,NaN,18.1,[{}],2024-11-01
9,2024,M10,October,NaN,18.2,[{}],2024-10-01
